#Mounting Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Libraries

In [ ]:
# Importing Libraries...
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity

# Loading Data

In [ ]:
# Loading Data...
ratings = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment_2/RAW_interactions.csv')
recipes = pd.read_csv('/content/drive/MyDrive/Semester-2/Recommendation System/Assignment_2/RAW_recipes.csv')

## Visulaizing Data

In [ ]:
ratings.head()

,user_id,recipe_id,date,rating,review
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall..."
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin..."


In [ ]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


In [ ]:
# Checking for duplicates
print(ratings.duplicated().sum())
print(recipes.duplicated().sum())

0
0


In [ ]:
print(ratings.shape)
print(recipes.shape)

(1132367, 5)
(231637, 12)


# Preprocessing

In [ ]:
# Replacing column name of recipes dataframe...
recipes.rename(columns={'id':'recipe_id'},inplace=True)
recipes.head()

,name,recipe_id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8


## Merging Data

In [ ]:
# Merging the ratings and recipes dataframe on recipe_id
ratings_with_name = ratings.merge(recipes, on='recipe_id')
ratings_with_name.head()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,38094,40893,2003-02-17,4,Great with a salad. Cooked on top of stove for...,white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
1,1293707,40893,2011-12-21,5,"So simple, so delicious! Great for chilly fall...",white bean green chile pepper soup,495,1533,2002-09-21,"['weeknight', 'time-to-make', 'course', 'main-...","[204.8, 5.0, 9.0, 26.0, 24.0, 2.0, 10.0]",4,"['combine beans , onion , chilies , 1 / 2 teas...",easy soup for the crockpot.,"['great northern beans', 'yellow onion', 'dice...",9
2,8937,44394,2002-12-01,4,This worked very well and is EASY. I used not...,devilicious cookie cake delights,20,56824,2002-10-27,"['30-minutes-or-less', 'time-to-make', 'course...","[132.3, 11.0, 39.0, 5.0, 4.0, 11.0, 5.0]",5,"['blend together cake mix , oil and eggs', 'ad...",NaN,"[""devil's food cake mix"", 'vegetable oil', 'eg...",4
3,126440,85009,2010-02-27,5,I made the Mexican topping and took it to bunk...,baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13
4,57222,85009,2011-10-01,5,"Made the cheddar bacon topping, adding a sprin...",baked potato toppings,10,64342,2004-02-25,"['15-minutes-or-less', 'time-to-make', 'course...","[2786.2, 342.0, 134.0, 290.0, 161.0, 301.0, 42.0]",3,['pick whichever topping you want to use and c...,these toppings sure makes a nice change from p...,"['mayonnaise', 'salsa', 'cheddar cheese', 'ref...",13


## Filtering based on users

**Reasoning:-**
* There are lots of users who have rated very less number of recipes and similar for the recipes.
* So, we have decided to take the genuine user and famous recipes only. 

In [ ]:
# Filtering merged dataframe based on the users...
X = ratings_with_name.groupby('user_id').count()['rating'] > 100
genuine_users = X[X].index
genuine_users

Int64Index([      1533,       1535,       2310,       2312,       3288,
                  4291,       4439,       4470,       4740,       5060,
            ...
               2206637,    2324285,    2399085,    2549237,    2597942,
            1800054678, 1802849661, 2000431901, 2000498330, 2001102678],
           dtype='int64', name='user_id', length=1368)

In [ ]:
# Filtered ratings...
filtered_ratings = ratings_with_name[ratings_with_name['user_id'].isin(genuine_users)]

## Filtering Based on Recipes

In [ ]:
# Filtering Based on recipes...
y = filtered_ratings.groupby('name').count()['rating']>=50
famous_recipes = y[y].index

In [ ]:
# Final ratings dataframe...
final_ratings = filtered_ratings[filtered_ratings['name'].isin(famous_recipes)]

In [ ]:
# Dropping Duplicates if any...
final_ratings.drop_duplicates()

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
4212,149363,96886,2005-05-11,5,This was awesome! Our whole family loved this ...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4214,209441,96886,2005-06-06,5,"Awesome IS the word for this, I made this for ...",kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4216,73836,96886,2005-09-07,5,This was scrumptious. I halved the recipe suc...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4217,160974,96886,2005-09-11,5,I'm a spinach lover but the rest of the family...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
4218,195175,96886,2005-09-15,5,This makes a great side dish for company. It's...,kittencal s spinach parmesan rice bake,40,89831,2004-08-04,"['60-minutes-or-less', 'time-to-make', 'course...","[351.2, 36.0, 3.0, 20.0, 35.0, 70.0, 5.0]",7,"['set oven to 350f', 'generously grease a 11 x...",this is a wonderful easy side dish to chicken ...,"['frozen chopped spinach', 'cooked white rice'...",13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1123827,130531,131018,2014-12-14,5,I have been making this recipe for a couple of...,pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4
1123829,121690,131018,2015-01-08,5,I&#039;ve made this twice without substitution...,pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4
1123855,51931,131018,2015-11-29,5,I am reviewing the recipe two ways. One as wr...,pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4
1123860,59686,131018,2016-03-18,5,"Since we butcher our own meat, I used a 4-1/2 ...",pulled pork crock pot,1110,50509,2005-07-25,"['time-to-make', 'course', 'main-ingredient', ...","[335.1, 10.0, 72.0, 24.0, 80.0, 10.0, 8.0]",26,"['slice one onion and place in crock pot', 'pu...","i found this pulled pork recipe years ago, and...","['pork roast', 'onions', 'ginger ale', 'barbec...",4


# Train-Test Split

In [ ]:
# Train-Test Split...
train_data, test_data = train_test_split(final_ratings, test_size=0.2)

In [ ]:
test_data

,user_id,recipe_id,date,rating,review,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
681238,118268,63860,2005-03-18,5,My 8 year old aspiring chef loves this. Quick...,breakfast in a mug,4,52595,2003-06-05,"['15-minutes-or-less', 'time-to-make', 'course...","[183.3, 25.0, 0.0, 13.0, 13.0, 46.0, 0.0]",12,"['place butter in 12 oz', 'microwave safe mug ...",i came across this recipe and had to give it a...,"['butter', 'milk', 'egg', 'pepper', 'salt', 'b...",7
593741,13593,32614,2007-05-20,5,I too could only find the Pillsburry Devil's F...,fudge crinkles a great 4 ingredient cake mix ...,15,37305,2002-06-30,"['15-minutes-or-less', 'time-to-make', 'course...","[110.8, 10.0, 26.0, 6.0, 2.0, 5.0, 4.0]",20,"['preheat oven to 350', 'stir dry cake mix , o...","these are chewy, fudgy, super easy cookies tha...","[""devil's food cake mix"", 'vegetable oil', 'eg...",4
188217,149363,200296,2008-12-11,5,Crazy delicious! I scaled this recipe down to ...,kittencal s easy and delicious ranch parmesan ...,40,89831,2006-12-12,"['60-minutes-or-less', 'time-to-make', 'course...","[486.3, 51.0, 8.0, 34.0, 59.0, 48.0, 5.0]",8,"['set oven to 400 degrees', 'in a shallow bowl...","there are plenty versions of this recipe, this...","['boneless skinless chicken breasts', 'dry bre...",8
838667,463763,261889,2012-02-20,5,I know your recipe says no substitutes with th...,kittencal s buttery cut out sugar cookies w i...,124,89831,2007-10-28,"['time-to-make', 'course', 'main-ingredient', ...","[144.2, 8.0, 50.0, 4.0, 3.0, 17.0, 7.0]",20,"['for cookies', 'in a large bowl combine butte...",you will not find a better sugar cookie anywhe...,"['butter', 'sugar', 'eggs', 'vanilla', 'almond...",13
884296,928301,37336,2012-02-15,5,wonderful. I love baking chicken! Just stick...,melt in your mouth chicken breasts,55,31695,2002-08-15,"['60-minutes-or-less', 'time-to-make', 'prepar...","[250.2, 17.0, 0.0, 10.0, 68.0, 21.0, 0.0]",3,"['mix mayonnaise or yogurt , cheese and season...",the mayonnaise make these chicken breast very ...,"['boneless chicken breast halves', 'mayonnaise...",6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65278,250031,18816,2007-04-07,4,Tasty way of preparing cabbage. I happen to *l...,cabbage for those who dislike cabbage,40,20371,2002-02-04,"['60-minutes-or-less', 'time-to-make', 'course...","[89.3, 9.0, 20.0, 2.0, 2.0, 13.0, 2.0]",7,"['in a large skillet , heat butter and oil ove...",this is so much nicer than plain steamed or bo...,"['butter', 'canola oil', 'green cabbage', 'bro...",8
17415,573325,216237,2011-11-16,5,"Mmm, FT, this was really, really tasty! Cheesy...",traditional english cheddar cauliflower cheese...,40,242729,2007-03-11,"['60-minutes-or-less', 'time-to-make', 'course...","[210.2, 20.0, 16.0, 7.0, 14.0, 41.0, 6.0]",13,"['if oven baking , pre-heat oven to 225c or 45...","when i was growing up, we never called this di...","['cauliflower', 'milk', 'mature cheddar cheese...",9
8330,32058,79944,2008-09-15,5,This was agred by all as a 5* recipe. I chang...,bacon wrapped chicken oamc,70,3288,2003-12-31,"['bacon', 'time-to-make', 'course', 'main-ingr...","[317.6, 33.0, 4.0, 14.0, 56.0, 51.0, 0.0]",11,"['flatten chicken to 1 / 2"" thickness', 'sprea...",this is another great way to make something gr...,"['boneless skinless chicken breast halves', 'c...",5
939429,138429,69173,2006-11-05,5,Delicious Kittencal! I used 11/2 lbs beef and ...,kittencal s italian melt in your mouth meatballs,50,89831,2003-08-20,"['60-minutes-or-less', 'time-to-make', 'course...","[1312.6, 129.0, 8.0, 108.0, 214.0, 174.0, 8.0]",5,['mix all ingredients together in a large bowl...,cooking the meatballs in simmering pasta sauce...,"['ground beef', 'egg', 'parmesan cheese', 'bre...",10


# Creating Pivot Table

In [ ]:
# create pivot table for train data
train_pt = train_data.pivot_table(index='name', columns='user_id', values='rating')
# calculate similarity scores
similarity_scores = cosine_similarity(train_pt.fillna(0))
# create pivot table for test data
test_pt = test_data.pivot_table(index='name', columns='user_id', values='rating')
# test_pt.fillna(0,inplace=True)
# train_pt.fillna(0,inplace=True)

In [ ]:
test_pt

user_id,1533,1535,2310,2312,3288,4291,4439,4470,4740,5060,...,1706426,1900301,1925885,2123645,2324285,2549237,1800054678,1802849661,2000431901,2001102678
name,,,,,,,,,,,,,,,,,,,,,
24k carrots,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4 minute spicy garlic shrimp,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40 minute hamburger buns,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
absolute best ever lasagna,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
all purpose stir fry sauce brown garlic sauce,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ww 0 point weight watchers cabbage soup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
yes virginia there is a great meatloaf,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN
yummy crispy baked fish,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
test_pt.columns

Int64Index([      1533,       1535,       2310,       2312,       3288,
                  4291,       4439,       4470,       4740,       5060,
            ...
               1706426,    1900301,    1925885,    2123645,    2324285,
               2549237, 1800054678, 1802849661, 2000431901, 2001102678],
           dtype='int64', name='user_id', length=1140)

# Recommendation Engine

In [ ]:
def get_top_k_recommendations(user_id, k, rated_items=[]):
    """
    Get top k recommendations for a given user ID.

    Arguments:
        user_id (int): ID of the user for whom to make recommendations.
        k (int): Number of top recommendations to return.
        rated_items (list): List of tuples of (recipe_name, rating) for recipes
            rated by the user in previous rounds. Default is an empty list.

    Returns:
        list: List of top k recommended recipe names for the user.
    """
    if user_id not in train_pt.columns:
        return []

    # update user's ratings with rated items from previous rounds
    for recipe_name, rating in rated_items:
        if recipe_name in train_pt.index:
            train_pt[user_id][recipe_name] = rating

    rated_recipes = train_pt[user_id]
    rated_recipes = rated_recipes[rated_recipes.notna()]
    unrated_recipes = train_pt.index.difference(rated_recipes.index)
    predicted_ratings = []

    for recipe_name in unrated_recipes:
        similarity_scores_for_recipe = similarity_scores[train_pt.index.get_loc(recipe_name)]
        user_ratings = train_pt[user_id].fillna(0)
        numerator = (user_ratings * similarity_scores_for_recipe).sum()
        denominator = similarity_scores_for_recipe.sum()

        if denominator == 0:
            predicted_rating = 0
        else:
            predicted_rating = numerator / denominator

        predicted_ratings.append((recipe_name, predicted_rating))
        
    top_recipe_names = [recipe_name for recipe_name, predicted_rating in sorted(predicted_ratings, key=lambda x: x[1], reverse=True)[:k]]
    return top_recipe_names

In [ ]:
def main():
  """
    Runs the main recommendation loop for the recipe recommender system.

    The loop prompts the user to enter their ID and displays the top k recommended
    recipes for the user based on their ratings and similarity to other users. The user
    can then select a recipe and provide a rating for it, and the loop will continue
    until the user is satisfied or there are no more recommendations left.

    Args:
        None

    Returns:
        None
    """
   
  # initialize list of rated items
  rated_items = []

  # Taking user_id from the user
  user_id = int(input("Enter user id:"))

  # loop until user is satisfied or there are no more recommendations
  while True:
      # get top k recommendations for user
      recommendations = get_top_k_recommendations(user_id, 5, rated_items)

      # check if there are any recommendations left
      if len(recommendations) == 0:
          print("No more recommendations")
          break

      # display recommendations to user
      print("Here are your recommendations:")
      for i, recommendation in enumerate(recommendations):
          print(f"{i+1}. {recommendation}")
      
      # ask user to select item they like or if they want more recommendations
      selection = input("Enter number of item you like or 'm' for more recommendations: ")

      # if user wants more recommendations, continue to next round
      if selection == 'm':
          continue

      # get the index of the selected item
      try:
          selection_idx = int(selection) - 1
          selected_item = recommendations[selection_idx]
      except:
          print("Invalid selection")
          continue

      # add selected item and rating to list of rated items
      rating = float(input(f"What rating would you give {selected_item} (0-5)? "))
      rated_items.append((selected_item, rating))

      # if user is satisfied, show ingredients of selected item
      if input("Are you satisfied? (y/n) ") == 'y':
          print(f"Ingredients for {selected_item}:")
          print(final_ratings[final_ratings['name'] == selected_item]['ingredients'].values[0])
          break

if __name__ == '__main__':
    main()

Enter user id:47559
Here are your recommendations:
1. easy peezy pizza dough  bread machine pizza dough
2. cracker barrel s hashbrowns casserole   copycat
3. dirty shrimp in butter beer sauce
4. ramen noodle egg foo yung
5. creamy corn salad
Enter number of item you like or 'm' for more recommendations: 1
What rating would you give easy peezy pizza dough  bread machine pizza dough (0-5)? 3
Are you satisfied? (y/n) n
Here are your recommendations:
1. cracker barrel s hashbrowns casserole   copycat
2. dirty shrimp in butter beer sauce
3. ramen noodle egg foo yung
4. creamy corn salad
5. sue b s chocolate cake
Enter number of item you like or 'm' for more recommendations: 4
What rating would you give creamy corn salad (0-5)? 5
Are you satisfied? (y/n) n
Here are your recommendations:
1. cracker barrel s hashbrowns casserole   copycat
2. ramen noodle egg foo yung
3. dirty shrimp in butter beer sauce
4. poverty meal
5. the sweetest blueberry muffins
Enter number of item you like or 'm' for 